In [1]:
using SplittingOrderConditions

In [2]:
s=8
p=5
@time begin    
    eqs = generate_equations(1,s, palindromic=true, with_brackets=false)
    for q=2:p
        merge!(eqs,generate_equations(q,s, palindromic=true, with_brackets=false)) 
    end
end    
eqs

  0.294642 seconds (577.99 k allocations: 39.528 MB, 1.11% gc time)


Dict{Array{Int64,1},AbstractString} with 8 entries:
  [0,1]       => "+2*b1*a2+2*b1*a3+2*b1*a4+2*b1*b4+2*b1*b3+2*b1*b2+2*b1*b1+2*b2…
  [0]         => "+1*a1+1*a2+1*a3+1*a4+1*b4+1*b3+1*b2+1*b1-1"
  [0,0,0,0,1] => "+5*b1*a2^4+20*b1*a2^3*a3+20*b1*a2^3*a4+20*b1*a2^3*b4+20*b1*a2…
  [0,0,1]     => "+3*b1*a2^2+6*b1*a2*a3+6*b1*a2*a4+6*b1*a2*b4+6*b1*a2*b3+6*b1*a…
  [0,0,1,0,1] => "+60*b1*a2*b2*a3^2+120*b1*a2*b2*a3*a4+120*b1*a2*b2*a3*b4+120*b…
  [0,0,0,1,1] => "+10*b1^2*a2^3+30*b1^2*a2^2*a3+30*b1^2*a2^2*a4+30*b1^2*a2^2*b4…
  [0,0,0,1]   => "+4*b1*a2^3+12*b1*a2^2*a3+12*b1*a2^2*a4+12*b1*a2^2*b4+12*b1*a2…
  [0,0,1,1]   => "+6*b1^2*a2^2+12*b1^2*a2*a3+12*b1^2*a2*a4+12*b1^2*a2*b4+12*b1^…

In [3]:
vars = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"]
funs = Dict{Array{Int64,1},Function}()
for lw in keys(eqs)
    for k =1:length(vars)
        eqs[lw] = replace(eqs[lw], vars[k], string("x(",k,")"))        
    end
    funs[lw] = SplittingOrderConditions.compile_fg(eqs[lw], length(vars))
end    
funs=collect(values(funs))

# of attemps to call form = 306
# of attemps to call form = 11
# of attemps to call form = 331
# of attemps to call form = 108
# of attemps to call form = 555
# of attemps to call form = 221
# of attemps to call form = 219
# of attemps to call form = 331


8-element Array{Function,1}:
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)

In [4]:
a = [0.0948061248011184,0.295318061899783,1.53754156461765,-1.76438699139276,
     0.0815411353469756,-0.000346187375210155,0.516036508037932,0.239489784064514]
b = [0.239489784064514,0.516036508037932,-0.000346187375210155,0.0815411353469756,
    -1.76438699139276,1.53754156461765,0.295318061899783,0.0948061248011184]
x = [BigFloat(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 3.284152125467809482767519642677476869662589806216058320636409027683003186193324e-14
res = 1.066634991358370061492688570360775527091885272177340293678612410515173065352047e-24
res = 6.520443917322062402922169467673567152184416441919793343723693167523111596720383e-48
res = 1.17451892349284446905254385334085434167177604956333426836322367194292848523009e-75


In [5]:
a

8-element Array{BigFloat,1}:
  9.480612480096383755810238332092734295342265842893021596275902029430768282124701e-02
  2.953180618997683333982799787452393320472145416348581713921729632789296702805809e-01
  1.537541564619954771866660112165026811200228013610318688766209981025206572878256    
 -1.764386991395374283204348438900170800972430834544106121236887781167578344571381    
  8.154113534711735851774025233958719758552038570752172710018665663195316858490895e-02
 -3.461873752067720546596206064633419451746355641025988198797095382700431672899566e-04
  5.16036508038544019149622640437979835447145581419329353984986889207173627040678e-01 
  2.394897840642327347686026924978736236840742893072505628504519802682776661329925e-01

In [6]:
b

8-element Array{BigFloat,1}:
  2.394897840642327347686026924978736236840742893072505628504519802682776661329925e-01
  5.16036508038544019149622640437979835447145581419329353984986889207173627040678e-01 
 -3.461873752067720546596206064633419451746355641025988198797095382700431672899566e-04
  8.154113534711735851774025233958719758552038570752172710018665663195316858490895e-02
 -1.764386991395374283204348438900170800972430834544106121236887781167578344571381    
  1.537541564619954771866660112165026811200228013610318688766209981025206572878256    
  2.953180618997683333982799787452393320472145416348581713921729632789296702805809e-01
  9.480612480096383755810238332092734295342265842893021596275902029430768282124701e-02

In [7]:
a=[0.201651044312325,0.562615975356585,0.253874038247542,-0.835351693190397,
    0.0680149460931738,-0.102733803148438,0.273128836056527,0.578800656272682]
b=[0.578800656272682,0.273128836056527,-0.102733803148438,0.0680149460931738,
    -0.835351693190397,0.253874038247542,0.562615975356585,0.201651044312325]
x = [BigFloat(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 1.883271295730021542957273486643548024825501147245366326589017518861298133585104e-15
res = 3.007912408428557697270705224694414107343315480010314519125487119802583882403688e-27
res = 1.389569169748473132922779612923049495867301989226660858474497314955700950408356e-52
res = 3.972637535343444527677721856888183802713360167640689437110903596277552229454716e-76


In [8]:
a

8-element Array{BigFloat,1}:
  2.016510443123242295010717537127204350343897695135273956068179634356826878006001e-01
  5.626159753565692004726136871495739306245002418928744167752083427967419233021117e-01
  2.538740382475548454088261616327640537887272103604989852329201422299961638614295e-01
 -8.353516931903706357129392355396398222058695622004032927093969848250637070847104e-01
  6.801494609316509176653714245969736234055991715893380050062743095376070430288747e-02
 -1.027338031484321420816894557883886342801172074332317945637108267769652758423583e-01
  2.731288360565244789123179237944314885888137612222677858196072939859034375087217e-01
  5.788006562726649317332620225788411861089958694855327033379266381999440661513172e-01

In [9]:
b

8-element Array{BigFloat,1}:
  5.788006562726649317332620225788411861089958694855327033379266381999440661513172e-01
  2.731288360565244789123179237944314885888137612222677858196072939859034375087217e-01
 -1.027338031484321420816894557883886342801172074332317945637108267769652758423583e-01
  6.801494609316509176653714245969736234055991715893380050062743095376070430288747e-02
 -8.353516931903706357129392355396398222058695622004032927093969848250637070847104e-01
  2.538740382475548454088261616327640537887272103604989852329201422299961638614295e-01
  5.626159753565692004726136871495739306245002418928744167752083427967419233021117e-01
  2.016510443123242295010717537127204350343897695135273956068179634356826878006001e-01

In [10]:
a=Complex{Float64}[0.0616859578475632 - 0.0222823670735455im,0.100252119833815 + 0.0665813505646068im,
    0.173445604000608 + 0.0645347825330634im,0.142290226977369 - 0.0867529559497755im,
    0.108004395790385 - 0.107731532365639im,0.180508075049188 - 0.00844653859129045im,
    0.126231681726252 + 0.126845335227947im,0.107581938774818 - 0.0327480743453665im]
b=Complex{Float64}[0.107581938774818 - 0.0327480743453665im,
    0.126231681726252 + 0.126845335227947im,0.180508075049188 - 0.00844653859129045im,
    0.108004395790385 - 0.107731532365639im,0.142290226977369 - 0.0867529559497755im,
    0.173445604000608 + 0.0645347825330634im,0.100252119833815 + 0.0665813505646068im,
    0.0616859578475632 - 0.0222823670735455im]
x = [Complex{BigFloat}(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(Complex{BigFloat},n,n)
f=zeros(Complex{BigFloat},n)
g=zeros(Complex{BigFloat},n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 9.351088856389245945865075297989778198569056097910885365085548448543841526082178e-15
res = 7.83776066642287586649210479063798498249465108352910611202340891644682250701444e-22
res = 1.294048616391037085011696879277947599505724606761281449776548050674539418794095e-41
res = 2.286229959766329520787732464520840544202940250352902781442375028177954129952915e-77


In [11]:
a

8-element Array{Complex{BigFloat},1}:
 6.168595784859109424379843422161445371422367031536680311317629391969514199290263e-02-2.228236707396364035949975968671562744923709082096192480324603659193214509684778e-02im
  1.00252119839293154352277026785837311496548538536478455172567661832231658414216e-01+6.658135056707195071948508981148362921443577049056794133051610895433609171026112e-02im
  1.73445604001127437122682376753370831303599352140055084824863577685824958080055e-01+6.453478253014967114573942989676273618816291601334291727780863420463680225705505e-02im
 1.422902269732557020533341214022138791440669214243052322164718960977419693026926e-01-8.675295595502738947058641357262101986267295779085090526630706063607479973831568e-02im
 1.080043957830595334777308033784256203158581849001575504324231584293991298551569e-01-1.077315323540064128413870530898261896293948381271446508518411971977603831770945e-01im
 1.805080750470063893413572056580201364732210819809652874391233168248962808320333e-01-8.446538597

In [12]:
b

8-element Array{Complex{BigFloat},1}:
 1.075819387789741967547922811641949048475227121994847997955774510450355136737389e-01-3.274807434427748379662523726069659700735896579959728170754567288579861083246104e-02im
 1.262316817286924926540277506363228627049595385031867870057966441651753478492105e-01+1.268453352278416445318635499224149685985053572106080328320760098157251685786566e-01im
 1.805080750470063893413572056580201364732210819809652874391233168248962808320333e-01-8.446538597788339928989606020801900052440191175964128811460785663132123701252231e-03im
 1.080043957830595334777308033784256203158581849001575504324231584293991298551569e-01-1.077315323540064128413870530898261896293948381271446508518411971977603831770945e-01im
 1.422902269732557020533341214022138791440669214243052322164718960977419693026926e-01-8.675295595502738947058641357262101986267295779085090526630706063607479973831568e-02im
  1.73445604001127437122682376753370831303599352140055084824863577685824958080055e-01+6.453478253